In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
from ipywidgets.widgets import Button
from ipywidgets.widgets import Layout, HBox, VBox
from IPython.display import display
import bqplot as bq
import time
import threading
import time

from tqdm.notebook import tqdm
import random
import gym
from gym import spaces
from adgroup import AdGroup
from advertisers import Advertiser
from asa_env import ASAEnv
from client import Client
from probability_distributions import ProbabilityDistributions

import ipywidgets as widgets


from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import DQN

from IPython.display import clear_output
import seaborn as sns


In [2]:
# Initialize AdGroup, Client, and ASAEnv objects based on slider values

prob_dist = ProbabilityDistributions()

ad_group = AdGroup(num_keywords=30, keyword_bids=0.5, relevancy_level=2)

keywords = ad_group.keyword_bids.keys()

client = Client(name="Client_1",
                avg_cpa=2,
                avg_ttr=0.1,
                avg_cvr=0.05,
                cpa_goal=0,
                budget=100,
                ad_group=ad_group)

auction_competitiveness_level = 2
target = 3

asa_env = ASAEnv(keywords, client, prob_dist, auction_competitiveness_level, target)

model_trained = False

def run_simulation(num_keywords, competitiveness, cpa_goal, target, keyword_bid, keyword_relevancy, daily_budget, loop):
    
    global model_trained
    model_on = 0
    
    if model_on == 1 and not model_trained:
        # train model
        env = DummyVecEnv([lambda: asa_env])

        # Initiate the agent
        model = DQN("MlpPolicy",
                    env,
                    verbose=0,
                    exploration_initial_eps=0.9,
                    exploration_fraction=0.7,
                   gamma=0.5)
        
        # Train the agent
        model.learn(total_timesteps=90)
        model.save("dqn_asa")
        model_trained = True
    
    
    env = asa_env

    
    env.client.budget = daily_budget
    env.client.cpa_goal = cpa_goal
    
    ad_group = AdGroup(num_keywords, keyword_bid, keyword_relevancy)
    env.client.ad_group = ad_group
    #env.client.ad_group.update_keyword_bid = keyword_bid

    keywords = ad_group.keyword_bids.keys()
    env.auction_competitiveness_level = competitiveness
    env.available_keywords = keywords
    env.target = target


    # Run the simulation and collect data for plotting
    obs = env.reset()
    done = False
    obs = np.array(obs).astype(float)
    
    
    while not done:
        if model_on == 1 and model_trained:
            model = DQN.load("dqn_asa")
            action, _states = model.predict(obs, deterministic=True)
        else:
            action = cpa_goal  # Define your action strategy
        obs, reward, done, _ = env.step(action)


        # Set a custom color palette
        custom_colors = ['#53f18c', '#ff9955', '#4488ff', '#ff88dd']

        # Set the background color
        plt.rcParams['axes.facecolor'] = '#232323'

        
        custom_style = {'axes.labelcolor': 'black',
                'xtick.color': 'black',
                'ytick.color': 'black'}
        
        sns.set_style("darkgrid", rc=custom_style)

        # Increase figure size
        plt.figure(figsize=(12, 8))

        # Collect data for plotting
        values_to_plot = 30
        cpa_history = env.hist.tail(values_to_plot)["avg_cpa"]
        cpa_goal_history = env.hist.tail(values_to_plot)["cpa_goal"]
        daily_spend_history = env.hist.tail(values_to_plot)["spend"]
        impressions_history = env.hist.tail(values_to_plot)["impressions"]
        days = env.hist.tail(values_to_plot)["day"]

        # Plotting in real-time
        plt.subplot(2, 2, 1)
        plt.plot(days, cpa_history, label='Avg CPA', color=custom_colors[0])
        plt.plot(days, cpa_goal_history, label='CPA Goal', color=custom_colors[1])
        plt.legend()
        plt.xlabel('Days')
        plt.ylabel('CPA')
        plt.title('Average CPA vs. CPA Goal')

        plt.subplot(2, 2, 2)
        plt.plot(days, daily_spend_history, label='Daily Spend', color=custom_colors[2])
        plt.legend()
        plt.xlabel('Days')
        plt.ylabel('Spend')
        plt.title('Daily Spend')

        plt.subplot(2, 2, 3)
        plt.plot(days, impressions_history, label='Impressions', color=custom_colors[3])
        plt.legend()
        plt.xlabel('Days')
        plt.ylabel('Impressions')
        plt.title('Impressions')

        # Adjust layout and spacing
        plt.tight_layout()

        # Show the plots
        plt.show()


/Users/moe/code/mabuhalala/AI/asa_ml_project/cpa_goals_ml/simulation_interactive/probability_distributions.py:10: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df_keyword = pd.read_csv("../raw_data/adobe_asa_keywords_sample.csv") #keyword_level()


1.0 1.0 1.0 1.0


In [3]:
widgets.interact(run_simulation,
                 num_keywords=(1, 50, 1),
                 competitiveness=(1, 3, 1),
                 cpa_goal=(0, 20, 1),
                 target=(0,10, 0.5),
                 keyword_bid=(0.1, 9, 0.5),
                 keyword_relevancy=(1, 3, 1),
                 daily_budget=(50, 600, 1),
                loop=widgets.Play(min=0, max=500)
               );

interactive(children=(IntSlider(value=25, description='num_keywords', max=50, min=1), IntSlider(value=2, descr…